In [ ]:
# Library imports

from __future__ import annotations

from dataclasses import dataclass, asdict
from pathlib import Path
import hashlib
import json
import numpy as np
import pandas as pd
import pyarrow.dataset as ps
import glob
import os
import matplotlib.pyplot as plt
import random
import time
import math
import logging
import itertools

from openpyxl import load_workbook


import xlsxwriter as xw

In [6]:
UPGRADE_NUM = "0"
STATE = "DC"

# Define the file pattern to search for the RESSTOCK *.Parquet files
DATA_DIR = "../data/"

# Define input data directories
INPUT_DATA_DIR = os.path.join(DATA_DIR, "input/")

# RESSTOCK directories
INPUT_DATA_DIR_RESSTOCK = os.path.join(INPUT_DATA_DIR, "resstock/")
INPUT_DATA_DIR_RESSTOCK_BUILDINGPROFILES = os.path.join(INPUT_DATA_DIR_RESSTOCK, "timeseries_individual_buildings/")

RESSTOCK_BUILDING_CHARACTERISTICS_FILE = 'DC_upgrade0.xlsx'
RESSTOCK_BUILDING_CHARACTERISTICS_FILEPATH = DATA_DIR + 'background/' + RESSTOCK_BUILDING_CHARACTERISTICS_FILE
building_characteristics = pd.read_excel(RESSTOCK_BUILDING_CHARACTERISTICS_FILEPATH, sheet_name='building_characteristics')

df_resstock_list = [pd.read_parquet(os.path.join(INPUT_DATA_DIR_RESSTOCK_BUILDINGPROFILES, f"upgrade={UPGRADE_NUM}/state={STATE}/" + str(resstock_filename) + "-0.parquet")) for resstock_filename in ['2553']]
   

In [31]:
resstock_list = pd.concat(df_resstock_list)
resstock_list.merge(building_characteristics, on='bldg_id', how='inner')#[['bldg_id', 'in.geometry_building_number_units_mf', 'in.sqft', 'out.electricity.total.energy_consumption..kwh']]


,bldg_id,timestamp,in.sqft,out.electricity.ceiling_fan.energy_consumption..kwh_x,out.electricity.clothes_dryer.energy_consumption..kwh_x,out.electricity.clothes_washer.energy_consumption..kwh_x,out.electricity.cooling.energy_consumption..kwh_x,out.electricity.cooling_fans_pumps.energy_consumption..kwh_x,out.electricity.dishwasher.energy_consumption..kwh_x,out.electricity.ev_charging.energy_consumption..kwh_x,...,calc.weighted.emissions_reduction.electricity.lrmer_high_re_cost_15..co2e_mmt,calc.weighted.emissions_reduction.electricity.lrmer_high_re_cost_25..co2e_mmt,calc.weighted.emissions_reduction.electricity.lrmer_high_re_cost_low_ng_price_15..co2e_mmt,calc.weighted.emissions_reduction.electricity.lrmer_high_re_cost_low_ng_price_25..co2e_mmt,calc.weighted.emissions_reduction.electricity.lrmer_low_re_cost_15..co2e_mmt,calc.weighted.emissions_reduction.electricity.lrmer_low_re_cost_25..co2e_mmt,calc.weighted.emissions_reduction.electricity.lrmer_low_re_cost_high_ng_price_15..co2e_mmt,calc.weighted.emissions_reduction.electricity.lrmer_low_re_cost_high_ng_price_25..co2e_mmt,calc.weighted.emissions_reduction.electricity.lrmer_mid_case_15..co2e_mmt,calc.weighted.emissions_reduction.electricity.lrmer_mid_case_25..co2e_mmt
0,2553,2018-01-01 00:15:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2553,2018-01-01 00:30:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2553,2018-01-01 00:45:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2553,2018-01-01 01:00:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2553,2018-01-01 01:15:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,2553,2018-12-31 23:00:00,854.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
35036,2553,2018-12-31 23:15:00,854.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
35037,2553,2018-12-31 23:30:00,854.0,0.00032,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
35038,2553,2018-12-31 23:45:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df_resstock_list[0]

,bldg_id,timestamp,in.sqft,out.electricity.ceiling_fan.energy_consumption..kwh,out.electricity.clothes_dryer.energy_consumption..kwh,out.electricity.clothes_washer.energy_consumption..kwh,out.electricity.cooling.energy_consumption..kwh,out.electricity.cooling_fans_pumps.energy_consumption..kwh,out.electricity.dishwasher.energy_consumption..kwh,out.electricity.ev_charging.energy_consumption..kwh,...,out.schedules.lighting_interior,out.schedules.no_space_cooling,out.schedules.no_space_heating,out.schedules.occupants,out.schedules.peak_period,out.schedules.plug_loads_other,out.schedules.plug_loads_tv,out.schedules.power_outage,out.schedules.pre_peak_period,out.schedules.vacancy
0,2553,2018-01-01 00:15:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0492,0.0,0.0,1.0,NaN,0.562,0.0159,0.0,NaN,0.0
1,2553,2018-01-01 00:30:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0492,0.0,0.0,1.0,NaN,0.562,0.0159,0.0,NaN,0.0
2,2553,2018-01-01 00:45:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0492,0.0,0.0,1.0,NaN,0.562,0.0159,0.0,NaN,0.0
3,2553,2018-01-01 01:00:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0492,0.0,0.0,1.0,NaN,0.562,0.0159,0.0,NaN,0.0
4,2553,2018-01-01 01:15:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0492,0.0,0.0,1.0,NaN,0.562,0.0159,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,2553,2018-12-31 23:00:00,854.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5050,0.0,0.0,1.0,NaN,0.779,0.5360,0.0,NaN,0.0
35036,2553,2018-12-31 23:15:00,854.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5050,0.0,0.0,1.0,NaN,0.779,0.5360,0.0,NaN,0.0
35037,2553,2018-12-31 23:30:00,854.0,0.00032,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4440,0.0,0.0,1.0,NaN,0.750,0.4670,0.0,NaN,0.0
35038,2553,2018-12-31 23:45:00,854.0,0.00237,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0492,0.0,0.0,1.0,NaN,0.562,0.0159,0.0,NaN,0.0
